In [82]:
import os
import numpy as np
import matplotlib.pyplot as plt
x_ai_path = (os.path.abspath(os.path.join("", os.pardir)))
intra_day_csv = os.path.join(x_ai_path, "Data","Data","Smard","IntraDay","IntraDayPreisindexIdAEPvonNetztransparenzDE.csv")
allData_csv = os.path.join(x_ai_path, "submission", "merged_data", "allData.csv")

import pandas as pd
df_all_data = pd.read_csv(allData_csv)
df_intra_day = pd.read_csv(intra_day_csv, delimiter=';', decimal=',')

Leider enthält die CSV der Intraday Daten wesentlich genauere Werte als unsere andere CSV, die nur ganze Stunden enthält. Außerdem enthält sie aktuell Werte über einen längeren Zeitraum, die beschnitten werden müssen. 

In [83]:
df_intra_day['(Uhrzeit) von'] = pd.to_datetime(df_intra_day['Datum von'] + ' ' + df_intra_day['(Uhrzeit) von'], format='%Y-%m-%d %H:%M')
df_intra_day_filtered = df_intra_day[df_intra_day['(Uhrzeit) von'].dt.minute == 0]
earliest_datetime = pd.Timestamp("2020-06-30 22:00:00")
cutoff_datetime = pd.Timestamp("2024-12-23 10:00:00")
df_intra_day_filtered = df_intra_day_filtered[df_intra_day_filtered['(Uhrzeit) von'] <= cutoff_datetime]
print(df_intra_day_filtered)

         Datum von       (Uhrzeit) von Zeitzone (Uhrzeit) bis Zeitzone.1  \
0       2020-06-30 2020-06-30 22:00:00      UTC         22:15        UTC   
4       2020-06-30 2020-06-30 23:00:00      UTC         23:15        UTC   
8       2020-07-01 2020-07-01 00:00:00      UTC         00:15        UTC   
12      2020-07-01 2020-07-01 01:00:00      UTC         01:15        UTC   
16      2020-07-01 2020-07-01 02:00:00      UTC         02:15        UTC   
...            ...                 ...      ...           ...        ...   
157088  2024-12-23 2024-12-23 06:00:00      UTC         06:15        UTC   
157092  2024-12-23 2024-12-23 07:00:00      UTC         07:15        UTC   
157096  2024-12-23 2024-12-23 08:00:00      UTC         08:15        UTC   
157100  2024-12-23 2024-12-23 09:00:00      UTC         09:15        UTC   
157104  2024-12-23 2024-12-23 10:00:00      UTC         10:15        UTC   

        ID AEP in €/MWh  
0                 43.75  
4                 33.22  
8        

Nun muss auch noch die große CSV gefiltert werden, um sich auf den gleichen Zeitraum zu beschränken

In [84]:
df_all_data['Date'] = pd.to_datetime(df_all_data['Date'], format='%Y-%m-%d %H:%M:%S')
earliest_datetime = pd.Timestamp("2020-06-30 22:00:00")
df_all_data_filtered = df_all_data[
    (df_all_data['Date'] >= earliest_datetime) & (df_all_data['Date'] <= cutoff_datetime)
]
print(df_all_data_filtered)

                     Date  E_load_forecast_MWh_x  day_ahead_prices_EURO_x  \
48190 2020-06-30 22:00:00                44015.0                    23.92   
48191 2020-06-30 23:00:00                41885.0                    25.04   
48192 2020-07-01 00:00:00                40509.0                    25.59   
48193 2020-07-01 01:00:00                40207.0                    25.03   
48194 2020-07-01 02:00:00                40883.0                    24.78   
...                   ...                    ...                      ...   
87462 2024-12-23 06:00:00                56771.0                    52.01   
87463 2024-12-23 07:00:00                60324.0                    59.34   
87464 2024-12-23 08:00:00                62624.0                    59.62   
87465 2024-12-23 09:00:00                63753.0                    52.73   
87466 2024-12-23 10:00:00                62947.0                    52.00   

       E_generation_forecast_MWh_x  E_solar_forecast_MWh_x_x  \
48190      

Now we do a cross correlation analysis

In [85]:
series1 = df_all_data_filtered['day_ahead_prices_EURO_x'].fillna(0)
series2 = df_intra_day_filtered['ID AEP in €/MWh'].fillna(0)

cross_corr_value = np.corrcoef(series1, series2)[0, 1]
print(cross_corr_value)

0.7184630651379397
